# Building a simulation with Orca from the ground up

Sam Maurer, UrbanSim User Meeting, Nov. 4, 2016

### Import Orca and its new testing framework

In [ ]:
import orca
import orca_test as ot
from orca_test import OrcaSpec, TableSpec, ColumnSpec

### Import NumPy and Pandas libraries, for data work

In [ ]:
import numpy as np
import pandas as pd

### Register some tables with our baseline data

In [ ]:
@orca.table(cache=True)
def parks():
    return pd.read_csv('data/parks.csv')

@orca.table(cache=True)
def zones():
    return pd.read_csv('data/zones.csv')

### Look at the data

In [ ]:
print orca.get_table('parks').to_frame()

In [ ]:
print orca.get_table('zones').to_frame()

### Register model steps for population, employment, budget growth

In [ ]:
@orca.step()
def zone_transition(zones):
    zones['pop'] *= [1.05, 1]
    zones['empl'] *= [1.05, 1]
    return
    
@orca.step()
def parks_transition(parks, year):
    if year <= 2018:
        parks['budget'] -= 1000
    return

### Register a model step to predict visitors to parks

In [ ]:
@orca.step()
def predict_visitors(parks, zones, year):
    
    # Calculate weighted accessibilities
    parks['pop_access'] = parks.acc_cbd * zones.pop[0] + parks.acc_res * zones.pop[1]
    parks['empl_access'] = parks.acc_cbd * zones.empl[0] + parks.acc_res * zones.empl[1]

    # Predict visitors
    vis = 0.15 * parks.pop_access + 1.5 * parks.empl_access + 0.5 * np.log(parks.budget)
    
    # Save the output
    parks['visitors' + str(year)] = np.round(vis)
    return

### Loose ends: Define the year variable

In [ ]:
@orca.injectable()
def year():
    return orca.get_injectable('iter_var')

### Test the baseline data (new Orca functionality!)

In [ ]:
def test_data():
    ot.assert_orca_spec(OrcaSpec('',
        TableSpec('zones',
            ColumnSpec('pop', min=0, missing=False),
            ColumnSpec('empl', min=0, missing=False)),
        TableSpec('parks',
            ColumnSpec('acc_cbd', min=0, missing=False),
            ColumnSpec('acc_res', min=0, missing=False),
            ColumnSpec('budget', min=0.01, missing=False))))

try:
    test_data()
    print "All clear!"

except Exception, e:
    print e

### Run the model !!!

In [ ]:
%%capture
orca.clear_cache()
years = range(2016, 2021)

orca.run(['zone_transition', 'parks_transition', 'predict_visitors'], iter_vars=years)

### Look at the output

In [ ]:
orca.get_table('parks').to_frame()